In [1]:
import json
import numpy as np
import cv2
from glob import glob
from scipy.spatial.transform import Rotation as R
from scipy.optimize import least_squares

In [2]:
def se3_log(R_mat, t_vec):
    """SE(3) 로그맵: 회전 행렬 + 이동 벡터 → 6D 벡터"""
    rvec, _ = cv2.Rodrigues(R_mat)
    return np.hstack([rvec.flatten(), t_vec.flatten()])

def se3_exp(xi):
    """SE(3) 지수맵: 6D 벡터 → 회전 행렬 + 이동 벡터"""
    rvec = xi[:3]
    tvec = xi[3:].reshape(3, 1)
    R_mat, _ = cv2.Rodrigues(rvec)
    return R_mat, tvec

def residual_icp(xi, T_cam2target_list, T_base2ee_list):
    R_cb, t_cb = se3_exp(xi)
    T_cb = np.eye(4)
    T_cb[:3, :3] = R_cb
    T_cb[:3, 3] = t_cb.flatten()

    residuals = []
    for T_cam2target, T_base2ee in zip(T_cam2target_list, T_base2ee_list):
        T_cam2target_pred = T_cb @ T_base2ee
        delta_T = np.linalg.inv(T_cam2target) @ T_cam2target_pred
        R_err = delta_T[:3, :3]
        t_err = delta_T[:3, 3].reshape(3, 1)
        residuals.append(se3_log(R_err, t_err))
    
    return np.concatenate(residuals)

def optimize_icp(T_cam2target_list, T_base2ee_list, T_cb_init):
    R_init = T_cb_init[:3, :3]
    t_init = T_cb_init[:3, 3].reshape(3, 1)
    xi0 = se3_log(R_init, t_init)

    result = least_squares(
        residual_icp, xi0, method='lm',
        args=(T_cam2target_list, T_base2ee_list)
    )

    R_opt, t_opt = se3_exp(result.x)
    T_cb_opt = np.eye(4)
    T_cb_opt[:3, :3] = R_opt
    T_cb_opt[:3, 3] = t_opt.flatten()
    return T_cb_opt

In [3]:
# --- SE(3) log/exp ---
def se3_log(R_mat, t_vec):
    rvec, _ = cv2.Rodrigues(R_mat)
    return np.hstack([rvec.flatten(), t_vec.flatten()])

def se3_exp(xi):
    rvec = xi[:3]
    tvec = xi[3:].reshape(3, 1)
    R_mat, _ = cv2.Rodrigues(rvec)
    return R_mat, tvec

def se3_log12(T):
    return se3_log(T[:3,:3], T[:3,3].reshape(3,1))

def se3_exp12(xi12):
    R_cb, t_cb = se3_exp(xi12[:6])
    R_et, t_et = se3_exp(xi12[6:])
    T_cb = np.eye(4); T_cb[:3,:3]=R_cb; T_cb[:3,3]=t_cb.flatten()
    T_et = np.eye(4); T_et[:3,:3]=R_et; T_et[:3,3]=t_et.flatten()
    return T_cb, T_et

# --- Residuals: 보드=EE 고정, T_ct_pred = T_cb @ T_be @ T_et ---
def residual_joint(xi12, T_cam2target_list, T_base2ee_list, 
                   trans_weight=20.0, weights=None):
    # trans_weight [1/m]: 0.05m(5cm) ≈ 1rad 정도로 맞추려면 ≈ 20
    T_cb, T_et = se3_exp12(xi12)
    res = []
    for i, (T_ct_obs, T_be) in enumerate(zip(T_cam2target_list, T_base2ee_list)):
        T_ct_pred = T_cb @ T_be @ T_et
        delta = np.linalg.inv(T_ct_obs) @ T_ct_pred
        R_err = delta[:3,:3]
        t_err = delta[:3,3].reshape(3,1)
        rvec, _ = cv2.Rodrigues(R_err)
        r = np.hstack([rvec.flatten(), (trans_weight * t_err).flatten()])
        if weights is not None:
            r *= weights[i]
        res.append(r)
    return np.concatenate(res)

def optimize_joint(T_cam2target_list, T_base2ee_list, 
                   T_cb_init, T_et_init=None,
                   trans_weight=20.0, weights=None):
    if T_et_init is None:
        T_et_init = np.eye(4)

    xi_cb0 = se3_log(T_cb_init[:3,:3], T_cb_init[:3,3].reshape(3,1))
    xi_et0 = se3_log(T_et_init[:3,:3], T_et_init[:3,3].reshape(3,1))
    xi0 = np.hstack([xi_cb0, xi_et0])

    result = least_squares(
        residual_joint, xi0, method='trf',
        loss='soft_l1', f_scale=1.0, max_nfev=400,
        args=(T_cam2target_list, T_base2ee_list, trans_weight, weights)
    )
    T_cb, T_et = se3_exp12(result.x)
    return T_cb, T_et, result

In [4]:
MAIN_PATH = "/home/ros/llm_robot/data/Calibration/Eye-to-Hand11"
with open(f"{MAIN_PATH}/cam2base.json", "r") as f:
    data = json.load(f)

R_cam2base = np.array(data["R_cam2base"])
t_cam2base = np.array(data["t_cam2base"]).reshape(3)
q_init = R.from_matrix(R_cam2base).as_quat()
x0 = np.hstack((q_init, t_cam2base))

In [5]:
sorted_pose_files = sorted(glob(f"{MAIN_PATH}/poses/*.json"))
sorted_charuco_files = sorted(glob(f"{MAIN_PATH}/*.json"))
len(sorted_pose_files), len(sorted_charuco_files)

(25, 27)

In [6]:
pose_data = []
for pf, cf in zip(sorted_pose_files, sorted_charuco_files[:25]):
    with open(pf) as f: pose = json.load(f)
    with open(cf) as f: charuco = json.load(f)
    
    R_base2ee = np.array(pose["R_base2ee"])
    t_base2ee = np.array(pose["t_base2ee"]).reshape(3)
    
    rvec = np.array(charuco["rvec_target2cam"]).reshape(3)
    tvec = np.array(charuco["tvec_target2cam"]).reshape(3)
    R_board2cam = R.from_rotvec(rvec).as_matrix()
    t_board2cam = tvec
    
    pose_data.append({
        "R_base2ee": R_base2ee,
        "t_base2ee": t_base2ee,
        "R_board2cam": R_board2cam,
        "t_board2cam": t_board2cam
    })

In [7]:
T_base2ee_list = []
T_cam2target_list = []

for d in pose_data:
    # --- base to end-effector ---
    T_be = np.eye(4)
    T_be[:3, :3] = d["R_base2ee"]
    T_be[:3, 3] = d["t_base2ee"]
    T_base2ee_list.append(T_be)
    
    # --- cam to target (solvePnP에서 board to cam → 역행렬) ---
    R_bc = d["R_board2cam"]
    t_bc = d["t_board2cam"].reshape(3, 1)

    T_bc = np.eye(4)
    T_bc[:3, :3] = R_bc
    T_bc[:3, 3] = t_bc.flatten()

    # cam2target = inverse(board2cam)
    T_ct = np.linalg.inv(T_bc)
    T_cam2target_list.append(T_ct)

In [8]:
T_cam2base_init = np.eye(4)
T_cam2base_init[:3, :3] = R_cam2base
T_cam2base_init[:3, 3] = t_cam2base

T_cam2base_icp_refined = optimize_icp(
    T_cam2target_list,
    T_base2ee_list,
    T_cb_init=T_cam2base_init
)
T_cam2base_icp_refined

array([[-0.00754092,  0.99988081, -0.01347241,  0.14904503],
       [ 0.9815216 ,  0.00997722,  0.19109106, -0.16658555],
       [ 0.1912027 , -0.01178246, -0.98147985,  0.51928934],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [9]:
# 결과 저장
if T_cam2base_icp_refined is not None:
    output = {
        "R_cam2base": T_cam2base_icp_refined[:3,:3].tolist(),
        "t_cam2base": T_cam2base_icp_refined[:3,3].tolist()
    }
    with open(f"{MAIN_PATH}/cam2base_icp.json", "w") as f:
        json.dump(output, f, indent=4)
    print("Optimized cam2base saved.")

else:
    print("RANSAC failed to find a valid model.")

Optimized cam2base saved.
